# Basic demo: teaching to copy (random characters)

In [ ]:
# import src to path
import sys
import os
sys.path.append('../src')

if __name__ == "__main__":
    # try get SLURM JOB ID
    try:
        job_id = os.environ['SLURM_JOB_ID']
    except:
        job_id = 'debug'
    logdir = f'logs/{job_id}'
    os.makedirs(logdir, exist_ok=True)

In [ ]:
import ciphers

## data gen

In [ ]:
import preprocessing
import torch.utils.data
dataset = preprocessing.generate_dataset(rows=2000, min_length=50, max_length=50)
# using torch .8, .1 .1

# apply cipher to get pairs
training_pairs = [(ciphers.nothing(t), t) for t in dataset]

train, dev, test = torch.utils.data.random_split(dataset, [1600, 200, 200])


In [ ]:
## actual training
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

tokenizer = transformers.ByT5Tokenizer()

# colla

